# Many time pad

- Let us see what goes wrong when a stream cipher key is used more than once. Below are eleven hex-encoded ciphertexts that are the result of encrypting eleven plaintexts with a stream cipher, all with the same stream cipher key. Your goal is to decrypt
the last ciphertext, and submit the secret message within it as solution.
- Hint: XOR the ciphertexts together, and consider what happens when a space is XORed with a character in a-z, A-Z

## TODO
- Add writeup - detailed explanation of the solution for this problem
- Descriptions for each function

In [1]:
NONEKEY = 0

def longest_length(my_list):
    running_max = 0
    for ct in my_list:
        if len(ct) > running_max:
            running_max = len(ct)
    return running_max


def truncated(a, b):
    length = min(len(a), len(b))
    return a[:length], b[:length]


def truncated3(a, b, c):
    length = min(len(a), len(b), len(c))
    return a[:length], b[:length], c[:length]


def convert_to_bytes(hexstring):
    r = []
    #Get two characters at a time
    for i in range(len(hexstring) // 2):
        snippet = hexstring[2*i:( 2*i + 2)]
        x = int(snippet, 16)
        r.append(x)
    return r


def xor_bytes(x, y):
    a, b = truncated(x, y)
    r =[]
    for i, j in zip(a, b):
        r.append(i ^ j)
    return r


def within_range(x):
    return 65 <= x <= 90 or 97 <= x <= 122


def decode_raw(cipher, key):
    
    r = ''
    c, k = truncated(cipher, key)
    
    for i, j in zip(c, k):
        if j == NONEKEY:
            r+="_"
            continue

        x = i ^ j
        r+=chr(x)
    return r 


def build_key(key, c1, c2, c3, debug=False):

    xm12 = xor_bytes(c1, c2)
    xm13 = xor_bytes(c1, c3)
    xm23 = xor_bytes(c2, c3)
    
    length = len(c1)
    space = ord(" ")
    
    for i in range(0, length):
        
        if key[i] != NONEKEY: 
            continue 
            
        if c1[i] == c2[i] or c1[i] == c3[i] or c2[i] == c3[i]:
            continue
        
        m12, m13, m23 = xm12[i], xm13[i], xm23[i]
            
        if within_range(m13) and within_range(m23):            
            key[i] = space ^ c3[i]
            #print("3: ", key[i], m13 ^ space ^ c1[i], m23 ^ space ^ c2[i])            
        elif within_range(m12) and within_range(m23):
            key[i] = space ^ c2[i]
            #print("2: ", key[i], m12 ^ space ^ c1[i], m23 ^ space ^ c3[i])            
        elif within_range(m12) and within_range(m13):
            key[i] = space ^ c1[i]
            #print("1: ", key[i], m12 ^ space ^ c2[i], m13 ^ space ^ c3[i])

In [2]:
################################################
# HELPERS FOR TESTS
################################################

def decode_bytes(x):
    r = ''
    for i in x:
        if within_range(i) or i == ord(" "):
            r+=chr(i)
        else:
            r+="_"
    return r


def decode(cipher, key):
    
    r = ''
    space = ord(" ")
    c, k = truncated(cipher, key)
    
    for i, j in zip(c, k):
        if j == NONEKEY:
            r+="_"
            continue

        x = i ^ j
        if within_range(x) or x == space:
            r+=chr(x)
        else:
            r+="_"
    return r 


################################################
# TESTS 
################################################

c1hex = "315c4eeaa8b5f8aaf9174145bf43"
c2hex = "234c02ecbbfbafa3ed18510abd11"
c3hex = "32510ba9a7b2bba9b8005d43a304"

ciphertexts = [c1hex, c2hex, c3hex]
cipherlength = len(ciphertexts)

# each byte is 2 characters in hex  
key_length = longest_length(ciphertexts) // 2
keybytes = [NONEKEY for i in range(key_length)]

c1, c2, c3 = truncated3(c1hex, c2hex, c3hex)

c1bytes = convert_to_bytes(c1)
c2bytes = convert_to_bytes(c2)
c3bytes = convert_to_bytes(c3)

x12 = xor_bytes(c1bytes, c2bytes)
x13 = xor_bytes(c1bytes, c3bytes)
x23 = xor_bytes(c2bytes, c3bytes)

answer12 = convert_to_bytes('12104c06134e5709140f104f0252')
answer13 = convert_to_bytes('030d45430f07430341171c061c47')
answer23 = convert_to_bytes('111d09451c49140a55180c491e15')

x12decoded = decode_bytes(x12)
x13decoded = decode_bytes(x13)
x23decoded = decode_bytes(x23)

build_key(keybytes, c1bytes, c2bytes, c3bytes)

message1 = decode(c1bytes, keybytes)
message2 = decode(c2bytes, keybytes)
message3 = decode(c3bytes, keybytes)

print(x12 == answer12)
print(x13 == answer13)
print(x23 == answer23)

print(x12decoded == "__L__NW____O_R") 
print(x13decoded == "__EC__C_A____G") 
print(x23decoded == "___E_I__U__I__") 


print(keybytes == [0, 0, 110, 137, 0, 219, 216, 0, 152, 0, 0, 42, 0, 99])

print(message1 == "__ c_n _a__o_ ")
print(message2 == "__le_ w_u__ _r")
print(message3 == "__e _ic_ __i_g")

True
True
True
True
True
True
True
True
True
True


In [3]:
from random import shuffle
from copy import deepcopy
import numpy
from scipy import stats

def parsed_ciphertext(filepath):
    
    ciphertexts = []

    with open(filepath) as f:
        for line in f:
            s = line.strip(' \t\n\r')
            if s != '':
                ciphertexts.append(s)
    
    return ciphertexts, ciphertexts[0] 


def find_key(ct, do_shuffling=True, modify_arg=False):
    
    ciphertexts = None 
    total = len(ct)
    
    if modify_arg is True: 
        ciphertexts = ct
    else:
        ciphertexts = deepcopy(ct)
    
    if do_shuffling is True:
        shuffle(ciphertexts)
    
    # each byte is 2 characters in hex
    keylength = longest_length(ciphertexts) // 2
    keybytes = [0 for i in range(keylength)]
    
    for i in range(total - 2):
        for j in range(i + 1, total - 1):
            for k in range(j + 1, total):
        
                c1hex, c2hex, c3hex = truncated3(
                    ciphertexts[i], ciphertexts[j], ciphertexts[k])
            
                c1 = convert_to_bytes(c1hex)
                c2 = convert_to_bytes(c2hex)
                c3 = convert_to_bytes(c3hex)

                build_key(keybytes, c1, c2, c3)
                
    return keybytes


def finalize_key(ks):
    keys = numpy.array(ks)
    modes = stats.mode(keys)
    return modes[0][0]

In [4]:
filepath="./data/many-time-pad-ciphertexts.txt"
number_of_shuffles = 50
ciphertexts, target = parsed_ciphertext(filepath)

targetbytes = convert_to_bytes(target)
possiblekeys = []

for _ in range(number_of_shuffles):
    keybytes = find_key(ciphertexts, do_shuffling=True, modify_arg=True)    
    #message = decode_raw(targetbytes, keybytes)
    #print(message)
    possiblekeys.append(keybytes)
    
keybytes = finalize_key(possiblekeys)

# The secret message is: When using a stream cipher, never use the key more than once
message = decode_raw(targetbytes, keybytes)
print(message)

The secuet message is: Whtn using a stream cipher, never use the key more than once
